# Create a sphere asset with variants for the display color

In [22]:
from pxr import Usd, UsdGeom

In [23]:
import os
usd_scene_dir = '..\usd_scenes'
usd_sphere_filename='sphere.usda'
usd_sphere_filepath = os.path.join(usd_scene_dir, usd_sphere_filename)

# Create a usda layer containing just the geometry of a ball

In [45]:
# Execute only if you have already created a stage and need to redo it.
del stage

In [46]:
# Create stage
stage = Usd.Stage.CreateNew(usd_sphere_filepath)

# Create a transform with a sphere primitive inside
sphere = stage.DefinePrim('/sphere', 'Xform')
geom = stage.DefinePrim('/sphere/geom', 'Sphere')

# Clear primitive display color so variants will be composited properly otherwise the value of the property 
# in the primitive always wins 

# Use UsdGeom.Sphere schema to access display color
sphereSchema = UsdGeom.Sphere(geom)
colorAttr = sphereSchema.GetDisplayColorAttr()
colorAttr.Set([(0,0,1)])
colorAttr.Clear()

# Use UsdGeom.Gprim schema to access display color
colorAttr = UsdGeom.Gprim.Get(stage, '/sphere/geom').GetDisplayColorAttr()
colorAttr.Clear()

# Set transform as default primitive in case this usda is referenced
stage.SetDefaultPrim(sphere)
color = sphereSchema.GetDisplayColorAttr()

# Save and show result
stage.GetRootLayer().Save()
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "sphere"
)

def Xform "sphere"
{
    def Sphere "geom"
    {
        color3f[] primvars:displayColor
    }
}




## Add variantset and variants

In [54]:
variantset_name_list = sphere.GetVariantSets().GetNames()
for variantset_name in variantset_name_list:
    print variantset_name
    variant_set = sphere.GetVariantSet(variantset_name)
    print variant_set

shadingVariant


In [59]:
# Create a variant set only if it does not exists
variantset_name_list = sphere.GetVariantSets().GetNames()
variantset_name = 'shadingVariant'
if variantset_name not in variantset_name_list:
    variant_set = sphere.GetVariantSets().AddVariantSet('shadingVariant')

# Create each variant only if it does not exists
variant_list = {'red':[(1,0,0)], 'blue':[(0,0,1)], 'green':[(0,1,0)]}
for variant_name, variant_color in variant_list.iteritems():
    if variant_name not in variant_set.GetVariantNames():
        variant_set.AddVariant(variant_name)
        
    # Modify color attribute in the context of the variant
    variant_set.SetVariantSelection(variant_name)
    with variant_set.GetVariantEditContext():
        colorAttr.Set(variant_color)

# Save and show result
stage.GetRootLayer().Save()
print stage.GetRootLayer().ExportToString()

#usda 1.0
(
    defaultPrim = "sphere"
)

def Xform "sphere" (
    variants = {
        string shadingVariant = "red"
    }
    prepend variantSets = "shadingVariant"
)
{
    def Sphere "geom"
    {
        color3f[] primvars:displayColor
    }
    variantSet "shadingVariant" = {
        "blue" {
            over "geom"
            {
                color3f[] primvars:displayColor = [(0, 0, 1)]
            }

        }
        "green" {
            over "geom"
            {
                color3f[] primvars:displayColor = [(0, 1, 0)]
            }

        }
        "red" {
            over "geom"
            {
                color3f[] primvars:displayColor = [(1, 0, 0)]
            }

        }
    }
}




In [60]:
!usdview {usd_sphere_filepath}